# Reading and calculations

In [ ]:
import h5py
import utils
file_path = "our_processing\ML_Data.h5"

def read_data(file_path, group_name, signal_name= "filtered_intervals"):
    with h5py.File(file_path, "r") as hdf:
        group = hdf[group_name]
        data = group[signal_name][()]
    return data

group_names = [
"19070921", "19072205", "19072214",
"19072938", "19072939", "19072940",
"19080106", "19080715", "19081506",
"19082406", "19090308", "19090320",
"19101607", "19101619", "19102102",
"19102103", "19102524", "19102622",
"19112609", "19120302", "19120323",
"19120704", "19120723", "19121303",
"19121735", "20010826", "20010827",
"20011712", "20050628", "20052606",
"20061729", "20092226", "20092535",
"20101424", "20101822", "20102029",
"20120116", "20120922", "20121033",
"20121716", "20121718", "20122932", "20123017"]

patient_arrays = {group_name: read_data(file_path, group_name) for group_name in group_names}

patient_arrays

In [ ]:
import pandas as pd
metrics_df = pd.DataFrame()

for id, signal in patient_arrays.items():
    individual_metrics = utils.patients_metrics(signal)
    individual_metrics['id'] = id
    print(individual_metrics)
    metrics_df = pd.concat([metrics_df, individual_metrics], ignore_index=True)

Save the processed metrics_df

Read the clinical indicators

In [ ]:
clin_indic = utils.df_from_excel('actionable_data/Clinical indicators.xlsx')
clin_indic.rename(columns={'number': 'id'}, inplace=True)
clin_indic['id'] = clin_indic['id'].astype(str)
display(clin_indic.columns)

Merge and save both

In [ ]:
# Join combined_metrics and clin_indic on 'id'
metrics_df["id"].astype(str)
merged_data = metrics_df.merge(clin_indic, on='id', how='inner')

# Display the merged dataframe
merged_data.head()

In [ ]:
merged_data.to_pickle('our_processing/Waj_05_02_2025.pkl')
merged_data.to_csv('our_processing/Waj_05_02_2025.csv', index=False)

merged_data.head(6)

# Analysis

Keep only individuals with no complications or diabetic neuropathy only.

In [1]:
import pandas as pd

data = pd.read_pickle('our_processing/Waj_05_02_2025.pkl')
data.head()

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,...,UMA (mg),UCr (g),UACR (mg/g),Diabetic Complications,Diabetic nephropathy,Diabetic retinopathy and cataract,Diabetic peripheral neuropathy,Coronary artery disease and cardiac insufficiency,Lower extremity atherosclerosis or stenosis,Carotid plaque
0,0,300,57.334978,38.317409,5.596107,82.370347,0.0,0.0,1.849318,0.030068,...,411.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,300,600,85.691235,33.612203,12.064343,74.785043,0.0,0.0,6.452692,0.023383,...,411.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,600,900,76.251431,30.704001,8.991826,73.642222,0.0,0.0,2.295485,0.020062,...,411.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,900,1200,51.471703,34.198269,12.680115,69.584228,0.0,0.0,4.614101,0.036760,...,411.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1200,1500,71.014530,63.691131,15.126050,71.547532,0.0,0.0,4.093889,0.033405,...,411.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
data = data[ (data["Diabetic Complications"] == 0)| (data["Diabetic peripheral neuropathy"] == 1)]
data["Diabetic peripheral neuropathy"] = data["Diabetic peripheral neuropathy"].astype(bool)

In [3]:
data['Condition'] = data['Diabetic peripheral neuropathy'].apply(lambda x: 'DPN' if x else 'Diabetes')
data.head()

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,...,UCr (g),UACR (mg/g),Diabetic Complications,Diabetic nephropathy,Diabetic retinopathy and cataract,Diabetic peripheral neuropathy,Coronary artery disease and cardiac insufficiency,Lower extremity atherosclerosis or stenosis,Carotid plaque,Condition
0,0,300,57.334978,38.317409,5.596107,82.370347,0.0,0.0,1.849318,0.030068,...,NaN,NaN,0.0,0.0,0.0,False,0.0,0.0,0.0,Diabetes
1,300,600,85.691235,33.612203,12.064343,74.785043,0.0,0.0,6.452692,0.023383,...,NaN,NaN,0.0,0.0,0.0,False,0.0,0.0,0.0,Diabetes
2,600,900,76.251431,30.704001,8.991826,73.642222,0.0,0.0,2.295485,0.020062,...,NaN,NaN,0.0,0.0,0.0,False,0.0,0.0,0.0,Diabetes
3,900,1200,51.471703,34.198269,12.680115,69.584228,0.0,0.0,4.614101,0.036760,...,NaN,NaN,0.0,0.0,0.0,False,0.0,0.0,0.0,Diabetes
4,1200,1500,71.014530,63.691131,15.126050,71.547532,0.0,0.0,4.093889,0.033405,...,NaN,NaN,0.0,0.0,0.0,False,0.0,0.0,0.0,Diabetes


In [18]:


data.rename(columns={'LF/HF Ratio': 'LF HF ratio'}, inplace=True)
indicator_columns = data.columns[2:15]
print(indicator_columns)

Index(['SDRR', 'RMSSD', 'pNN50 (%)', 'Mean HR (bpm)', 'ULF Power',
       'ULF Peak Frequency', 'VLF Power', 'VLF Peak Frequency', 'LF Power',
       'LF Peak Frequency', 'HF Power', 'HF Peak Frequency', 'LF HF ratio'],
      dtype='object')


We plot all the images as html within the folder of [html_plots](html_plots)

In [ ]:
from population_metric_plotter import population_metric_plotter
for column in indicator_columns:
    population_metric_plotter(data, column, 'Condition')

We can then run a t_test for two independent means.

In [35]:
from scipy.stats import ttest_ind

result_df = pd.DataFrame()
for column in indicator_columns:
    group1 = data[data['Condition'] == 'Diabetes']
    group2 = data[data['Condition'] == 'DPN']

    t_stat, p_val = ttest_ind(group1[column], group2[column], nan_policy='omit')
    column_results = {'metric': column, 't-statistic': t_stat, 'p-value': p_val, 'p<0.05': p_val<0.05}
    result_df = pd.concat([result_df, pd.DataFrame([column_results])], ignore_index=True)
result_df.index = result_df.metric

pd.options.display.float_format = '{:.2e}'.format

result_df.columns.name = 'Student T Test'
result_df.drop(columns=['metric'], inplace=True)

display("First Group: Diabetes")
display("Second Group: DPN")
display(result_df)



'First Group: Diabetes'

'Second Group: DPN'

Student T Test,t-statistic,p-value,p<0.05
metric,,,
SDRR,2.05e+01,1.60e-90,True
RMSSD,1.31e+01,7.84e-39,True
pNN50 (%),1.39e+01,2.51e-43,True
Mean HR (bpm),-1.60e+01,2.22e-56,True
ULF Power,NaN,NaN,False
ULF Peak Frequency,NaN,NaN,False
VLF Power,1.06e+01,5.60e-26,True
VLF Peak Frequency,5.62e+00,1.97e-08,True
LF Power,1.25e+01,1.53e-35,True


In [34]:
from scipy.stats import mannwhitneyu

result_df = pd.DataFrame()
for column in indicator_columns:
    group1 = data[data['Condition'] == 'Diabetes']
    group2 = data[data['Condition'] == 'DPN']

    u_stat, p_val = mannwhitneyu(group1[column], group2[column], alternative='two-sided')
    column_results = {'metric': column, 'U-statistic': u_stat, 'p-value': p_val, 'p<0.05': p_val<0.05}
    result_df = pd.concat([result_df, pd.DataFrame([column_results])], ignore_index=True)
result_df.index = result_df.metric

result_df.columns.name = 'Mann Whitney U Test'
result_df.drop(columns=['metric'], inplace=True)

pd.options.display.float_format = '{:.2e}'.format

display("First Group: Diabetes")
display("Second Group: DPN")
display(result_df)

'First Group: Diabetes'

'Second Group: DPN'

Mann Whitney U Test,U-statistic,p-value,p<0.05
metric,,,
SDRR,4.54e+06,2.23e-130,True
RMSSD,4.10e+06,1.11e-63,True
pNN50 (%),4.00e+06,1.83e-52,True
Mean HR (bpm),2.11e+06,6.95e-64,True
ULF Power,3.10e+06,1.00e+00,False
ULF Peak Frequency,3.10e+06,1.00e+00,False
VLF Power,4.66e+06,1.66e-153,True
VLF Peak Frequency,3.39e+06,1.04e-06,True
LF Power,4.73e+06,1.47e-166,True


In [ ]:
group1 = data[data['Condition'] == 'Diabetes']
group2 = data[data['Condition'] == 'DPN']

In [37]:
group1.describe()

,t_start,t_end,SDRR,RMSSD,pNN50 (%),Mean HR (bpm),ULF Power,ULF Peak Frequency,VLF Power,VLF Peak Frequency,...,LDL-C (mmol/L),UMA (mg),UCr (g),UACR (mg/g),Diabetic Complications,Diabetic nephropathy,Diabetic retinopathy and cataract,Coronary artery disease and cardiac insufficiency,Lower extremity atherosclerosis or stenosis,Carotid plaque
count,5.66e+03,5.66e+03,5.66e+03,5.66e+03,5.66e+03,5.66e+03,5.66e+03,5.66e+03,5.66e+03,5.66e+03,...,4.48e+03,4.51e+03,3.93e+03,3.93e+03,5.66e+03,5.66e+03,5.66e+03,5.66e+03,5.66e+03,5.66e+03
mean,3.96e+04,3.99e+04,5.22e+01,3.43e+01,9.65e+00,7.50e+01,0.00e+00,0.00e+00,3.80e+00,2.82e-02,...,2.81e+00,9.75e+01,1.22e+01,7.05e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
std,2.40e+04,2.40e+04,3.01e+01,2.88e+01,1.44e+01,1.28e+01,0.00e+00,0.00e+00,7.70e+00,7.47e-03,...,8.65e-01,1.19e+02,5.65e+00,1.08e+01,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
min,0.00e+00,3.00e+02,5.61e+00,4.72e+00,0.00e+00,4.85e+01,0.00e+00,0.00e+00,6.40e-03,3.33e-03,...,8.60e-01,2.93e+00,3.89e+00,5.34e-01,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
25%,1.92e+04,1.95e+04,3.09e+01,1.64e+01,5.20e-01,6.53e+01,0.00e+00,0.00e+00,5.74e-01,2.34e-02,...,2.35e+00,9.68e+00,6.32e+00,1.11e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
50%,3.84e+04,3.87e+04,4.47e+01,2.60e+01,2.97e+00,7.41e+01,0.00e+00,0.00e+00,1.44e+00,3.01e-02,...,2.83e+00,3.76e+01,1.25e+01,1.77e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
75%,5.85e+04,5.88e+04,6.56e+01,4.30e+01,1.24e+01,8.39e+01,0.00e+00,0.00e+00,3.57e+00,3.35e-02,...,3.45e+00,1.45e+02,1.54e+01,9.32e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00
max,9.69e+04,9.72e+04,2.85e+02,3.60e+02,7.98e+01,1.35e+02,0.00e+00,0.00e+00,1.13e+02,3.69e-02,...,4.17e+00,4.11e+02,2.23e+01,3.92e+01,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00


In [ ]:
group2.describe()